## Openimagesv7+CTG

In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from einops import rearrange
from gem import create_gem_model, get_gem_img_transform, get_tokenizer


# =====================================
# torch.load wrapper
# =====================================
_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return _torch_load(*args, **kwargs)
torch.load = torch_load_wrapper

C:\Users\Konyang\miniconda3\envs\clip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Konyang\miniconda3\envs\clip\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "ViT-B/16"
pretrained = "laion400m_e32"

gem_model = create_gem_model(model_name=model_name, pretrained=pretrained, device=device)
gem_model.eval()

gem_img_transform = get_gem_img_transform()
tokenizer = gem_model.tokenizer

In [ ]:
class CTG(nn.Module):
    def __init__(self,
                 pos_ratio: float = 0.1,   # 상위 k% (양성 시드)
                 neg_ratio: float = 0.1,   # 하위 k% (음성 시드)
                 alpha: float = 0.5,       # 원로짓과 CTG 로짓 혼합 가중치
                 tau: float = 9.0,         # 유사도 스케일(temperature)
                 iters: int = 1):          # 반복 적용 횟수
        super().__init__()
        self.pos_ratio = pos_ratio
        self.neg_ratio = neg_ratio
        self.alpha = alpha
        self.tau = tau
        self.iters = max(1, iters)

    @torch.no_grad()
    def forward(self, feat_hwxd, logits_hwxc):
        """
        feat_hwxd  : [B, H, W, D]
        logits_hwxc: [B, H, W, C]
        """
        B, H, W, D = feat_hwxd.shape
        _, _, _, C = logits_hwxc.shape
        cur = logits_hwxc

        for _ in range(self.iters):
            scores = cur.permute(0, 3, 1, 2).reshape(B, C, H * W)

            k_pos = max(1, int(self.pos_ratio * H * W))
            k_neg = max(1, int(self.neg_ratio * H * W))

            feat = F.normalize(feat_hwxd.reshape(B, H * W, D), dim=-1)

            pos_proto = torch.zeros(B, C, D, device=feat.device)
            neg_proto = torch.zeros(B, C, D, device=feat.device)

            top_vals, top_idx = torch.topk(scores, k=k_pos, dim=-1, largest=True, sorted=False)
            bot_vals, bot_idx = torch.topk(scores, k=k_neg, dim=-1, largest=False, sorted=False)

            for b in range(B):
                for c in range(C):
                    pos_feats = feat[b, top_idx[b, c]]
                    neg_feats = feat[b, bot_idx[b, c]]
                    pos_proto[b, c] = F.normalize(pos_feats.mean(dim=0, keepdim=True), dim=-1)
                    neg_proto[b, c] = F.normalize(neg_feats.mean(dim=0, keepdim=True), dim=-1)

            sim_pos = torch.einsum("bnd,bcd->bnc", feat, pos_proto)
            sim_neg = torch.einsum("bnd,bcd->bnc", feat, neg_proto)
            ctg_score = self.tau * (sim_pos - sim_neg)

            ctg_score = ctg_score.reshape(B, H, W, C)
            cur = (1.0 - self.alpha) * cur + self.alpha * ctg_score

        return cur

class CTGWrapper(nn.Module):
    def __init__(self, patch_size=16, ctg_cfg=None):
        super().__init__()
        self.patch_size = patch_size
        self.ctg = CTG(**(ctg_cfg or {}))

    @torch.no_grad()
    def refine(self, feat_tokens_bnd, H, W, logits_bchw, mask_shape):
        """
        feat_tokens_bnd : [B, h*w, D]
        logits_bchw     : [B, C, h, w]
        mask_shape      : (H_out, W_out)
        """
        B, C, h, w = logits_bchw.shape
        feat_hwxd = feat_tokens_bnd.reshape(B, h, w, -1)

        logits_hwxc = logits_bchw.permute(0, 2, 3, 1).contiguous()
        refined_hwxc = self.ctg(feat_hwxd, logits_hwxc)

        refined_bchw = refined_hwxc.permute(0, 3, 1, 2).contiguous()
        refined_bchw = F.interpolate(refined_bchw, size=mask_shape, mode='bilinear')
        return refined_bchw

In [ ]:
patch_size = 16
ctg_cfg = dict(pos_ratio=0.10, neg_ratio=0.10, alpha=0.3, tau=9.0, iters=1)
ctg_wrapper = CTGWrapper(patch_size=patch_size, ctg_cfg=ctg_cfg)

In [ ]:
@torch.no_grad()
def gem_forward_with_ctg(image_bchw, texts):
    B, _, H, W = image_bchw.shape

    feat_gem, _ = gem_model.model.visual(image_bchw.to(device))
    feat_gem = F.normalize(feat_gem, dim=-1)

    h, w = H // patch_size, W // patch_size
    patch_tokens = feat_gem[:, 1:]

    tokenized = tokenizer(texts).to(device)
    text_emb = gem_model.model.encode_text(tokenized)
    text_emb = F.normalize(text_emb, dim=-1)
    text_emb = text_emb.unsqueeze(0).repeat(B, 1, 1)

    logits_init = 100.0 * torch.einsum("bnd,bcd->bnc", patch_tokens, text_emb)
    logits_init = rearrange(logits_init, "b (hh ww) c -> b c hh ww", hh=h, ww=w)

    logits_out = ctg_wrapper.refine(
        feat_tokens_bnd=patch_tokens,
        logits_bchw=logits_init,
        out_size_hw=(H, W)
    )

    return logits_out

In [ ]:
def compute_point_miou_for_image(logits, points_df, image_w, image_h, class_list):

    logits = logits[0]
    C, H_out, W_out = logits.shape

    pred_masks = {}
    for idx, cls in enumerate(class_list):
        prob = torch.sigmoid(logits[idx])
        pred_masks[cls] = (prob > 0.5).cpu().numpy()

    intersection = np.zeros(C)
    union = np.zeros(C)

    class_to_idx = {cls: i for i, cls in enumerate(class_list)}

    for _, row in points_df.iterrows():

        label_id = row["LabelName"]
        if label_id not in label_to_name:
            continue

        gt_class = label_to_name[label_id]
        if gt_class not in class_to_idx:
            continue

        gx = int((row["X"] / image_w) * W_out)
        gy = int((row["Y"] / image_h) * H_out)

        gx = max(0, min(W_out - 1, gx))
        gy = max(0, min(H_out - 1, gy))

        for cls, idx in class_to_idx.items():
            pred_pos = pred_masks[cls][gy, gx]
            gt_pos = (cls == gt_class)

            if gt_pos or pred_pos:
                union[idx] += 1
            if gt_pos and pred_pos:
                intersection[idx] += 1

    valid = union > 0
    if valid.sum() == 0:
        return 0.0, intersection, union, class_to_idx

    miou = (intersection[valid] / (union[valid] + 1e-6)).mean()
    return miou, intersection, union, class_to_idx

In [ ]:
image_root = r"C:\Users\Konyang\Desktop\open-vocab 데이터\openimagesv7\data"

class_csv = r"C:\Users\Konyang\Desktop\open-vocab 데이터\openimagesv7\oidv7-class-descriptions-points (1).csv"
anno_csv  = r"C:\Users\Konyang\Desktop\open-vocab 데이터\openimagesv7\oidv7-val-annotations-point-labels.csv"

class_df = pd.read_csv(class_csv)
anno_df_raw = pd.read_csv(anno_csv)

anno_df = anno_df_raw.rename(columns={
    "ImageId": "ImageID",
    "Label": "LabelName",
    "X": "X",
    "Y": "Y"
})

label_to_name = dict(zip(class_df["LabelName"], class_df["DisplayName"]))

In [ ]:
# =====================================
# p-mIoU 전체 평가
# =====================================
all_image_ids = anno_df["ImageID"].unique()
total_images = len(all_image_ids)

CHUNK_SIZE = 500
SAVE_PATH = "pmIoU_results_ctg_openclip.npy"

if os.path.exists(SAVE_PATH):
    miou_results = list(np.load(SAVE_PATH, allow_pickle=True))
    start_index = len(miou_results)
    print(f"기존 진행 발견 → {start_index}개 완료됨. 이어서 진행.")
else:
    miou_results = []
    start_index = 0

class_intersection = {}
class_union = {}

print(f"총 평가 이미지 수: {total_images}")


for start in range(start_index, total_images, CHUNK_SIZE):

    end = min(start + CHUNK_SIZE, total_images)
    chunk_ids = all_image_ids[start:end]

    print(f"\n=== [{start} ~ {end}] {len(chunk_ids)}개 이미지 평가중... ===")

    for image_id in tqdm(chunk_ids, desc=f"Chunk {start}-{end}"):

        img_path = os.path.join(image_root, image_id + ".jpg")

        if not os.path.exists(img_path):
            miou_results.append(None)
            continue

        point_rows = anno_df[anno_df["ImageID"] == image_id]
        gt_label_ids = point_rows["LabelName"].unique()

        class_list = [label_to_name[l] for l in gt_label_ids if l in label_to_name]
        if len(class_list) == 0:
            miou_results.append(None)
            continue

        img_pil = Image.open(img_path).convert("RGB")
        W, H = img_pil.size

        img_tensor = gem_img_transform(img_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            logits_ctg = gem_forward_with_ctg(img_tensor, class_list)

        miou, inter, union, class_to_idx = compute_point_miou_for_image(
            logits_ctg, point_rows, W, H, class_list
        )

        miou_results.append(miou)

        # 클래스별 누적
        for cls, idx in class_to_idx.items():
            if cls not in class_intersection:
                class_intersection[cls] = 0.0
                class_union[cls] = 0.0
            class_intersection[cls] += inter[idx]
            class_union[cls] += union[idx]

    # 청크 저장
    np.save(SAVE_PATH, np.array(miou_results, dtype=object))
    print(f"✓ 청크 저장 완료 ({len(miou_results)}개 완료됨)")


# =====================================
# 최종 결과 출력
# =====================================
valid_scores = [m for m in miou_results if m is not None]
final_miou = np.mean(valid_scores)

print("\n====================================")
print(f" Final Dataset p-mIoU (CTG) = {final_miou*100:.4f}")
print("====================================")

print("\n========== 클래스별 mIoU ==========")
for cls in sorted(class_intersection.keys()):
    if class_union[cls] > 0:
        score = class_intersection[cls] / (class_union[cls] + 1e-6)
        print(f"{cls:30s} : {score*100:.4f}")

총 평가 이미지 수: 36702

=== [0 ~ 500] 500개 이미지 평가중... ===


Chunk 0-500: 100%|███████████████████████████████████████████████████████████████████| 500/500 [01:41<00:00,  4.92it/s]


✓ 청크 저장 완료 (500개 완료됨)

=== [500 ~ 1000] 500개 이미지 평가중... ===


Chunk 500-1000: 100%|████████████████████████████████████████████████████████████████| 500/500 [01:39<00:00,  5.01it/s]


✓ 청크 저장 완료 (1000개 완료됨)

=== [1000 ~ 1500] 500개 이미지 평가중... ===


...

✓ 청크 저장 완료 (36000개 완료됨)

=== [36000 ~ 36500] 500개 이미지 평가중... ===


Chunk 36000-36500: 100%|██████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 17144.66it/s]


✓ 청크 저장 완료 (36500개 완료됨)

=== [36500 ~ 36702] 202개 이미지 평가중... ===


Chunk 36500-36702: 100%|██████████████████████████████████████████████████████████| 202/202 [00:00<00:00, 12599.63it/s]

✓ 청크 저장 완료 (36702개 완료됨)

 Final Dataset p-mIoU (CTG) = 0.4738

========== 클래스별 mIoU ==========
Abdomen                        : 0.0417
Accordion                      : 0.2535
Acrylic paint                  : 0.2500
Action figure                  : 0.3235
Adapter (Electrical)           : 0.1172
Adhesive                       : 0.1194
Adhesive tape                  : 0.2258
Adult                          : 0.2143
Advertising                    : 0.1062
Agate                          : 0.0714
Agave                          : 0.5135
Agriculture                    : 0.2097
Airboat                        : 0.3514
Aircraft                       : 0.2048
Aircraft carrier               : 0.0192
Airline                        : 0.1818
Airplane                       : 0.1488
Airport                        : 0.1739
Airship                        : 0.3134
Alarm clock                    : 0.3933
Alcoholic beverage             : 0.1797
Algae                          : 0.4167
All-terrain vehicle      